# 1. Import Dependecies

In [64]:
import cv2
import pickle
import cvzone
import numpy as np

# 2. Parking Space Picking and Export Position List

In [65]:
WIDTH, HEIGHT = 107, 48
posList = []

In [66]:
def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:
        posList.append((x,y))
    if events ==cv2.EVENT_RBUTTONDOWN:
        for idx, pos in enumerate(posList):
            x1, y1 = pos
            if x1 < x < x+WIDTH and y1 < y < y+HEIGHT:
                posList.pop(idx)
    

In [ ]:
while True:
    img = cv2.imread('carParkImg.png')#dışarıda olursa statik olur ve resimden dikdörtgenler silinmez
    
    #column 1
    #cv2.rectangle(img, (50,90),(160,670), (0,0,255), 2)
    #cv2.rectangle(img, (165,90),(270,670), (0,0,255), 2)

    for pos in posList:
        cv2.rectangle(img, pos,(pos[0]+WIDTH, pos[1]+HEIGHT), (0,0,255), 2)

    cv2.imshow("Image", img)
    cv2.setMouseCallback("Image", mouseClick)
    cv2.waitKey(1)


In [68]:
cv2.destroyAllWindows()

In [69]:
'''img = cv2.imread('carParkImg.png')

while True:
    #column 1
    #cv2.rectangle(img, (50,90),(160,670), (0,0,255), 2)
    #cv2.rectangle(img, (165,90),(270,670), (0,0,255), 2)

    #column 2
    #cv2.rectangle(img, (50,90),(160,670), (0,0,255), 2)
    #cv2.rectangle(img, (165,90),(270,670), (0,0,255), 2)

    for pos in posList:
        cv2.rectangle(img, pos,(pos[0]+WIDTH, pos[1]+HEIGHT), (0,0,255), 2)

    cv2.imshow("Image", img)
    cv2.setMouseCallback("Image", mouseClick)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()'''

'img = cv2.imread(\'carParkImg.png\')\n\nwhile True:\n    #column 1\n    #cv2.rectangle(img, (50,90),(160,670), (0,0,255), 2)\n    #cv2.rectangle(img, (165,90),(270,670), (0,0,255), 2)\n\n    #column 2\n    #cv2.rectangle(img, (50,90),(160,670), (0,0,255), 2)\n    #cv2.rectangle(img, (165,90),(270,670), (0,0,255), 2)\n\n    for pos in posList:\n        cv2.rectangle(img, pos,(pos[0]+WIDTH, pos[1]+HEIGHT), (0,0,255), 2)\n\n    cv2.imshow("Image", img)\n    cv2.setMouseCallback("Image", mouseClick)\n    if cv2.waitKey(0) & 0xFF == ord(\'q\'):\n        break\n\ncv2.destroyAllWindows()'

In [118]:
with open('CarParkPos', 'wb') as f:
    pickle.dump(posList, f)

# 3. Load Position List and Load Video

In [70]:
posList = []

with open('CarParkPos', 'rb') as f:
    posList = pickle.load(f)

In [71]:
while True:
    img = cv2.imread('carParkImg.png')
    
    for pos in posList:
        cv2.rectangle(img, pos,(pos[0]+WIDTH, pos[1]+HEIGHT), (0,0,255), 2)

    cv2.imshow("Image", img)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [72]:
#video feed
cap = cv2.VideoCapture('carPark.mp4')

while cap.isOpened():
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT): #(current frame == total frame)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0) #current frame set 0

    success, img = cap.read()

    for pos in posList:
        cv2.rectangle(img, pos,(pos[0]+WIDTH, pos[1]+HEIGHT), (0,0,255), 2)
        
    cv2.imshow('Video', img)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# 4. Check Parking Space 

In [73]:
def checkParkingSpace(imgProcess):

    spaceCounter = 0

    for pos in posList:
        x,y = pos
        imgCrop = imgProcess[y:y+HEIGHT, x:x+WIDTH]
        #cv2.imshow(str(x*y), imgCrop)  
        count = cv2.countNonZero(imgCrop)

        if count < 900:
            color = (0,255,0)
            spaceCounter += 1
        else:
            color = (0,0,255)

        cvzone.putTextRect(img, str(count), (x,y+HEIGHT - 10), 1, 1, offset=0, colorR=color )
        cv2.rectangle(img, pos,(pos[0]+WIDTH, pos[1]+HEIGHT), color, 2)

    cvzone.putTextRect(img, f'{spaceCounter}/{len(posList)}', (30, 70), 5, 3, offset=20, colorR=(0,0,0))

In [74]:
threshold = 0

def updateThreshold(t):
    threshold = t

In [75]:
cap = cv2.VideoCapture('carPark.mp4')

#create Trackbar
cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars", 500, 45)
cv2.createTrackbar("Threshold", "TrackBars", 0, 50, updateThreshold)

while cap.isOpened():
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT): #(current frame == total frame)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0) #current frame set 0

    success, img = cap.read()
    
    #get trackbar threshold
    threshold = cv2.getTrackbarPos("Threshold", "TrackBars") # trackbar check best params

    #img process
    cropImg = img.copy() #kesilmiş resimlerde çizgiler gözükmesin diye 
    imgGray = cv2.cvtColor(cropImg, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3,3), 1)
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, threshold)
    imgMedian = cv2.medianBlur(imgThreshold, 5)
    kernel = np.ones((3,3), np.uint8)
    imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)

    #check parking space
    checkParkingSpace(imgDilate)

    cv2.imshow('Video', img)
    #cv2.imshow('Dilate Video', imgDilate)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [76]:
def imagePreprocess(img):
    cropImg = img.copy()
    imgGray = cv2.cvtColor(cropImg, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3,3), 1)
    imgThreshold = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16) #best params
    imgMedian = cv2.medianBlur(imgThreshold, 5)
    kernel = np.ones((3,3), np.uint8)
    imgDilate = cv2.dilate(imgMedian, kernel, iterations=1)
    return imgDilate

# 5. Test Real Time Video

In [77]:
cap = cv2.VideoCapture('carPark.mp4')

while cap.isOpened():
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT): #(current frame == total frame)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0) #current frame set 0

    success, img = cap.read()

    #img process
    imgDilate = imagePreprocess(img)
    
    #check parking space
    checkParkingSpace(imgDilate)

    cv2.imshow('Video', img)
    #cv2.imshow('Dilate Video', imgDilate)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()